In [2]:
import os
import sys

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
#import mplscience
import seaborn as sns

import scanpy as sc
import scvelo as scv
import scvi
from scvelo.plotting.simulation import compute_dynamics
from velovi import preprocess_data, VELOVI
from velovi._model import _compute_directional_statistics_tensor

Global seed set to 0


In [18]:
adata2 = scv.read("/home/liyr/hpz/real_data_graph/Forebrain/veloVI.h5ad")
adata2

AnnData object with n_obs × n_vars = 1720 × 822
    obs: 'clusters', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'velocity_s_self_transition'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes', 'velocity_s_gamma', 'velocity_s_qreg_ratio', 'velocity_s_r2'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'clusters_colors', 'log1p', 'neighbors', 'pca', 'velocity_params', 'velocity_s_graph', 'velocity_s_graph_neg', 'velocity_s_params'
    obsm: 'X_pca', 'X_umap', 'velocity_s_umap'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'ambiguous', 'matrix', 'spliced', 'unspliced', 'variance_velocity_s', 'velocity', 'velocity_s', 'velocity_u'
    obsp: 'connectivities', 'distances'

In [19]:
def gene_rank(adata, vkey="velocities_velovi"):
    from scipy.stats import rankdata
    scv.tl.velocity_graph(adata, vkey=vkey)
    tm = scv.utils.get_transition_matrix(
        adata, vkey=vkey, use_negative_cosines=True, self_transitions=True
    )
    tm.setdiag(0)
    adata.layers["Ms_extrap"] = tm @ adata.layers["Ms"]
    adata.layers["Ms_delta"] = adata.layers["Ms_extrap"] - adata.layers["Ms"]

    prod = adata.layers["Ms_delta"] * adata.layers[vkey]
    ranked = rankdata(prod, axis=1)
    adata.layers["product_score"] = prod
    adata.layers["ranked_score"] = ranked
    return (adata)

In [20]:
adata2 = gene_rank(adata2,"velocity_s")

computing velocity graph (using 1/52 cores)
    finished (0:00:02) --> added 
    'velocity_s_graph', sparse matrix with cosine correlations (adata.uns)


In [31]:
list_df = []
for i in ['Immature Neuron','Neuroblast','Neuron','Radial Glia']:
    cell_subset = adata2.obs.query("clusters == "+"'" + i +"'").index
    sub_cluster_data = adata2[cell_subset]
    sub_cluster_data.var['mean_product_score'] = sub_cluster_data.layers['product_score'].mean(axis=0)
    df = pd.DataFrame(
            {
                'Velocity coherence': sub_cluster_data.var['mean_product_score'],
                'Cell type': i
            }
        )
    list_df.append(df)


In [34]:
df_res = pd.concat(list_df)
df_res.to_csv("/home/liyr/zxc/top_like_genes/res/veloVI.csv")